In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, BartTokenizerFast, BartForConditionalGeneration
import torch
import pandas as pd
from tqdm import tqdm
from datasets import load_dataset

/home/ucloud/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ucloud/.local/lib/python3.12/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/ucloud/.local/lib/python3.12/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/ucloud/.local/lib/python3.12/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _to

In [3]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


nllb_model_name = "facebook/nllb-200-3.3B"
nllb_tokenizer = AutoTokenizer.from_pretrained(nllb_model_name)
nllb_model = AutoModelForSeq2SeqLM.from_pretrained(nllb_model_name).to(device)


bart_model_name = "s-nlp/bart-base-detox"
bart_tokenizer = BartTokenizerFast.from_pretrained(bart_model_name)
bart_model = BartForConditionalGeneration.from_pretrained(bart_model_name).to(device)


lang_id_mapping = {
    "zh": "zho_Hans",
    "en": "eng_Latn"
}


def translate_batch(texts, src_lang, tgt_lang):
    nllb_tokenizer.src_lang = lang_id_mapping[src_lang]
    inputs = nllb_tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
    tgt_lang_id = nllb_tokenizer.lang_code_to_id[lang_id_mapping[tgt_lang]]
    outputs = nllb_model.generate(**inputs, forced_bos_token_id=tgt_lang_id)
    return nllb_tokenizer.batch_decode(outputs, skip_special_tokens=True)


def detoxify_batch(texts):
    inputs = bart_tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
    outputs = bart_model.generate(**inputs)
    return bart_tokenizer.batch_decode(outputs, skip_special_tokens=True)


ds = load_dataset("textdetox/multilingual_paradetox", split="zh")
zh_toxic = ds["toxic_sentence"]
langs = ["zh"] * len(zh_toxic)


en_translated = []
batch_size = 8
for i in tqdm(range(0, len(zh_toxic), batch_size), desc="Translating to English"):
    batch = zh_toxic[i:i+batch_size]
    en_translated.extend(translate_batch(batch, src_lang="zh", tgt_lang="en"))


en_detox = []
for i in tqdm(range(0, len(en_translated), batch_size), desc="Detoxifying English"):
    batch = en_translated[i:i+batch_size]
    en_detox.extend(detoxify_batch(batch))


zh_detox = []
for i in tqdm(range(0, len(en_detox), batch_size), desc="Backtranslating to Chinese"):
    batch = en_detox[i:i+batch_size]
    zh_detox.extend(translate_batch(batch, src_lang="en", tgt_lang="zh"))


result_df = pd.DataFrame({
    "toxic_sentence": zh_toxic,
    "neutral_sentence": zh_detox,
    "lang": langs
})


result_df.to_csv("backtranslation_result.tsv", sep="\t", index=False)
print("Done. Saved to backtranslation_result.tsv")

/home/ucloud/.local/lib/python3.12/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Detoxifying English:   0%|          | 0/50 [00:00<?, ?it/s]/home/ucloud/.local/lib/python3.12/site-packages/transformers/generation/utils.py:1355: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Backtranslating to Chinese: 100%|██████████| 50/50 [00:13<00:00,  3.71it/s]

Done. Saved to backtranslation_result.tsv
